In [3]:
# =============================================================================
# Plain-Vanilla LoReFT Fine-Tuning (no Bits&Bytes, no 4-bit)
#
# This script:
# 1) Loads two TSVs (slang vs. non-slang) from your Windows paths
# 2) Builds a HuggingFace Dataset, splits into train/val/test
# 3) Tokenizes using RoBERTa-large (you can switch to roberta-base or another model)
# 4) Loads the model in full precision (fp32), wraps it with a LoReFT adapter at layer 15
# 5) Fine-tunes only that tiny adapter (all other weights stay frozen)
# 6) Reports validation & test accuracy
#
# USAGE:
#   python fine_tune_reft.py
#
# Make sure your Python interpreter (e.g. in PyCharm) is using the conda env
# where you installed torch, transformers, datasets, evaluate, pyreft, pandas.
# =============================================================================

import os
import random

import torch
import numpy as np
import pandas as pd

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import evaluate

import pyreft
from pyreft import ReftConfig, LoreftIntervention, get_reft_model

# ------------------------------
# 1) Check Device & Fix Seeds
# ------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}  |  CUDA available: {torch.cuda.is_available()}")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ------------------------------
# 2) Define Paths to Your TSV Files
# ------------------------------
SLANG_FILE     = r"C:\Users\yozev\PycharmProjects\Probing-Slang-Ambiguity-in-LLM\opensub\data\slang_OpenSub_filtered.tsv"
NON_SLANG_FILE = r"C:\Users\yozev\PycharmProjects\Probing-Slang-Ambiguity-in-LLM\opensub\data\slang_OpenSub_negatives_filtered.tsv"

if not os.path.isfile(SLANG_FILE):
    raise FileNotFoundError(f"Cannot find file: {SLANG_FILE}")
if not os.path.isfile(NON_SLANG_FILE):
    raise FileNotFoundError(f"Cannot find file: {NON_SLANG_FILE}")

# ------------------------------
# 3) Load TSVs into pandas & Build HF Dataset
# ------------------------------
df_pos = pd.read_csv(SLANG_FILE, sep="\t", dtype=str)
df_neg = pd.read_csv(NON_SLANG_FILE, sep="\t", dtype=str)

# Ensure each has a column named exactly "SENTENCE"
if "SENTENCE" not in df_pos.columns or "SENTENCE" not in df_neg.columns:
    raise KeyError("Both TSVs must have a column named 'SENTENCE'.")

# Assign labels: 1 = slang, 0 = non-slang
df_pos["label"] = 1
df_neg["label"] = 0

# Concatenate and shuffle
df = pd.concat([df_pos, df_neg], ignore_index=True)
df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Build a HuggingFace Dataset with columns ["text", "label"]
hf_dataset = Dataset.from_pandas(
    df[["SENTENCE", "label"]].rename(columns={"SENTENCE": "text"})
)

# Split 80% train / 10% val / 10% test
split1 = hf_dataset.train_test_split(test_size=0.2, seed=SEED)                # 80% train, 20% temp
split2 = split1["test"].train_test_split(test_size=0.5, seed=SEED)            # split 20% into 10/10

dataset = DatasetDict(
    train=split1["train"],
    validation=split2["train"],
    test=split2["test"],
)

print("Dataset sizes:")
print(f"  Train:      {len(dataset['train'])}")
print(f"  Validation: {len(dataset['validation'])}")
print(f"  Test:       {len(dataset['test'])}")

# ------------------------------
# 4) Tokenize with a Pretrained Tokenizer
# ------------------------------
MODEL_ID = "roberta-large"    # ← you can switch to "roberta-base" if GPU memory is tight

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

# Data collator to pad to multiples of 8 tokens (not strictly necessary here, but okay)
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

# Inspect a tokenized example
print("\nOne tokenized example:")
print(tokenized["train"][0])

# ------------------------------
# 5) Load Base Model (fp32) & Wrap with LoReFT
# ------------------------------
# 5.1) Load RoBERTa-large in FP32 (no quantization)
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID).to(device)

# 5.2) Fix classification head for 2 labels
base_model.config.num_labels = 2
base_model.config.id2label   = {0: "non-slang", 1: "slang"}
base_model.config.label2id   = {"non-slang": 0, "slang": 1}

# 5.3) Build a LoReFT intervention at layer 15's "output"
#      RoBERTa-large has 24 layers (0..23); we pick layer 15 arbitrarily.
embed_dim = base_model.config.hidden_size
reft_cfg = ReftConfig(
    representations={
        "layer": 15,
        "component": "output",   # RoBERTa's per-layer "output" tensor
    },
    intervention=LoreftIntervention(
        embed_dim=embed_dim,
        low_rank_dimension=4,     # Only 4 units in the low-rank adapter
    )
)

# 5.4) Wrap base_model with ReFT (this freezes all original weights except the LoReFT adapter)
model = get_reft_model(base_model, reft_cfg).to(device)

# 5.5) Print trainable vs. frozen parameter summary
print("\nTrainable vs. Frozen parameter counts:")
print(model.print_trainable_parameters())

# ------------------------------
# 6) Set Up Trainer & TrainingArguments
# ------------------------------
# 6.1) Load accuracy metric
acc_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"]}

# 6.2) Define TrainingArguments
training_args = TrainingArguments(
    output_dir                  = "./roberta_reft_finetune",
    per_device_train_batch_size = 8,       # if you OOM, lower to 4 or 2
    per_device_eval_batch_size  = 8,
    gradient_accumulation_steps = 1,
    learning_rate               = 2e-5,
    num_train_epochs            = 3,
    evaluation_strategy         = "epoch",
    save_strategy               = "epoch",
    logging_steps               = 50,
    fp16                        = True,    # fp16 is fine because model is full-precision
    report_to                   = "none",  # disable wandb/tensorboard
    load_best_model_at_end      = True,
    metric_for_best_model       = "accuracy",
)

# 6.3) Instantiate Trainer
trainer = Trainer(
    model            = model,
    args             = training_args,
    train_dataset    = tokenized["train"],
    eval_dataset     = tokenized["validation"],
    tokenizer        = tokenizer,
    data_collator    = data_collator,
    compute_metrics  = compute_metrics,
)

# ------------------------------
# 7) Train & Evaluate
# ------------------------------
if __name__ == "__main__":
    print("\n***** Starting fine-tuning *****\n")
    trainer.train()

    print("\n***** Evaluating on test set *****")
    metrics = trainer.evaluate(tokenized["test"])
    print(f"Test accuracy: {metrics['eval_accuracy']:.4f}")


ImportError: cannot import name 'insecure_hashlib' from 'huggingface_hub.utils' (C:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\utils\__init__.py)

In [2]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/50/9e/acf04ff375b0b49a45511c55d188bcea5c942da2aaf293096676110086d1/torch-2.7.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.10.0 from https://files.pythonhosted.org/packages/69/e0/552843e0d356fbb5256d21449fa957fa4eff3bbc135a74a691ee70c7c5da/typing_extensions-4.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for sympy>=1.13.3 from https://files.pythonhosted.org/packages/a2/09/77d55d46fd61b4a135c444fc97158ef34a095e5681d0a6c10b75bf356191/sympy-1.14.0-py3-none-any.whl.metadata
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB 640.0 kB/s eta 0:05:38
   ---------------------------------------- 0.1/216.1 MB 812.7 kB/s eta 0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
clip 1.0 requires torchvision, which is not installed.
open-clip-torch 2.23.0 requires torchvision, which is not installed.
timm 1.0.12 requires torchvision, which is not installed.
vit-pytorch 1.9.1 requires torchvision, which is not installed.
accelerate 1.7.0 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.15.1 which is incompatible.
albumentations 1.4.3 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
peft 0.8.0 requires huggingface-hub>=0.17.0, but you have huggingface-hub 0.15.1 which is incompatible.
pyreft 0.1.0 requires fsspec>=2024.2.0,